In [1]:
import os
os.chdir('../')

In [2]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareCallBackConfig:
    root_dir:Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:
from chicken_disease_classifier.constants import *

from chicken_disease_classifier.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def prepare_call_back_config(self)->PrepareCallBackConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_root_log_dir)])
        prepare_callback_config=PrepareCallBackConfig(
            root_dir=Path(config.root_dir),
            
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath))
        return prepare_callback_config

In [5]:
#Component
#Components for loading data
import os
from pathlib import Path
import urllib.request as request
import zipfile
import tensorflow as tf
import time
class PrepareCallback:
    def __init__(self,config:PrepareCallBackConfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [9]:
config=ConfigurationManager()
prepare_cb_config=config.prepare_call_back_config()
prepare_callback=PrepareCallback(config=prepare_cb_config)
callback_list=prepare_callback.get_tb_ckpt_callbacks()

[2023-08-06 16:05:06,030:INFO:common:yaml file:config\config.yaml loaded successfully]
[2023-08-06 16:05:06,032:INFO:common:yaml file:params.yaml loaded successfully]
[2023-08-06 16:05:06,033:INFO:common:created directory at:artifacts]
[2023-08-06 16:05:06,042:INFO:common:created directory at:artifacts\prepare_callbacks\checkpoint_dir]
[2023-08-06 16:05:06,042:INFO:common:created directory at:artifacts\prepare_callbacks\tensorboard_log_dir]


AttributeError: 'WindowsPath' object has no attribute 'endswith'